In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time

In [ ]:
SCROLL_PAUSE_TIME = 0.5

In [ ]:
def collect_sentences(responses, sentences, conversation_id):
    print("Conversation ID: {}".format(conversation_id))
    
#     for index, row in sentences.iterrows():
#         if row["conversation_id"] == conversation_id:
#             sentences.loc[index, "sentence"] = responses[index][(row["start_index"] - 1) : 
#                                                                 (row["end_index"] - 1)]
#             print(sentences.loc[index, "sentence"])

In [ ]:
# NOTE: may need to remove \n characters depending on semantics of how they parsed sentences. Also
# may need to replace the unicode vestigial characters with empty string instead of space.

def scrape_url(conversation_url):
    print("Conversation URL: {}".format(conversation_url))
        
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("headless")

    driver = webdriver.Chrome(options = chrome_options)
    driver.get(conversation_url + "?sort_by=oldest")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")

    while last_height != new_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        last_height = new_height

    soup = BeautifulSoup(driver.page_source)
    
    original_post = soup.find("div", {"id" : "subject_msg"})
    post_list = soup.find("div", {"id" : "post_show_" + conversation_url.split("/")[-1]})
    post_contents = post_list.find_all("div", {"class" : "resp_body"})
    post_user_data = post_list.find_all("div", {"class" : "username"})
    
#     responses = [original_post.text.replace(u"\xa0", u" ").strip()]
#     for response in :
#         responses.append(response.text.replace(u"\xa0", u" ").strip()

In [ ]:
def scrape_conversations(disease):
    conversations = pd.read_csv("gold_standard/conversations/" + disease + ".tsv", delimiter = "\t")
    posts = pd.read_csv("gold_standard/posts/" + disease + ".tsv", delimiter = "\t")
    expected_posts = posts.groupby("conversation_id")["post_id"].count()
    sentences = pd.read_csv("gold_standard/sentences/" + disease + ".tsv", delimiter = "\t")
    
    for index, row in conversations.iterrows():
        responses = scrape_url(row["url"])
        break

In [ ]:
def driver():
    diseases = pd.read_csv("gold_standard/diseases/diseases.tsv", delimiter = "\t")
    
    for disease in diseases["disease_id"]:
        print("Disease ID: {}".format(disease))
        scrape_conversations(disease)
        break

In [ ]:
driver()